# Space-varying meteo files on curvilinear grid for Delft3D

https://github.com/dact221/delft3d-write-meteo

**Authors:** Diego A. Casas (Federal University of Paraná, Brazil); Sofia M. G. Rocha (Lancaster University, UK)

**Version:** 20250804

**What's new?:** add a section to create spatialized input for radiation and wind magnitude in WAQ (segment function).

**Info:** this code can write up to 1.2 GB per minute. Writing all 7 meteo files for a 1.5-year-long time series with 10-minute data and 4 decimal places took about 13 minutes for the example .grd file in an SSD (16.7 GB).

**Warning:** writing large files in cloud folders (e.g., Google Drive, OneDrive, Dropbox) sometimes results in slow writing speeds because the cloud sofware tries to index and sync the content as it is being written. Prefer running this code in a local folder.

## Preamble

In [22]:
import re
import numpy as np
import pandas as pd
from osgeo import ogr # conda install gdal
import scipy.io as spio

In [2]:
# header: see Delft3D manual
METEO_HEADER_TPL = """FileVersion = 1.03
filetype = meteo_on_curvilinear_grid
NODATA_value = -999.000
grid_file = {grid_file}
first_data_value = {first_data_value}
data_row = {data_row}
n_quantity = 1
quantity1 = {quantity1}
unit1 = {unit1}
"""

# units and file extensions
METEO_UNITS_EXT = {
    'relative_humidity': ("%", "amr"),
    'air_temperature': ("Celsius", "amt"),
    'air_pressure': ("Pa", "amp"),
    'cloudiness': ("%", "amc"),
    'sw_radiation_flux': ("W m-2", "ams"),
    'x_wind': ("m s-1", "amu"),
    'y_wind': ("m s-1", "amv")
}

In [3]:
def read_grd_file(file):
    """Read x and y coordinate arrays (M by N) from .grd file.

    This function was adapted from https://github.com/Carlisle345748/Delft3D-Toolbox
    """
    with open(file, "r") as f:
        data = f.read()
    # read headers
    header = {}
    coordinate_system = re.search(r'Coordinate System = ([\w]+)', data)
    header['Coordinate System'] = coordinate_system.group(1) if coordinate_system else None
    missing_value = re.search(r'Missing Value\s+=\s+([\w+-.]+)', data)
    header['Missing Value'] = float(missing_value.group(1)) if missing_value else 0
    mn = re.search(r'\n\s+([\d]+)\s+([\d]+)\n', data)
    m = int(mn.group(1))
    n = int(mn.group(2))
    header['MN'] = [m, n]
    # read coordinates
    x = []
    y = []
    pattern = r' ETA=\s+\d+(\s+[\d.Ee+-]+\n?){' + str(m) + '}'
    matches = re.finditer(pattern, data)
    for index, match in enumerate(matches):
        cor = match[0].split()[2:]
        cor = [float(num) for num in cor]
        if index < n:
            x.extend(cor)
        else:
            y.extend(cor)
    x = np.array(x)
    y = np.array(y)
    # reshape to the original format
    x = x.reshape(n, m)
    y = y.reshape(n, m)
    return x, y

In [4]:
def is_fpv(x, y, polys):
    """Return a boolean mask array representing the presence of FPV-covered grid points.

    `polys` must be an iterable of osgeo.ogr.Geometry objects from a shapefile.
    """
    def is_in_polys(x, y):
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(x, y)
        for poly in polys:
            if point.Within(poly):
                return True
        return False
    is_in_polys = np.vectorize(is_in_polys)
    return is_in_polys(x, y)

## Input

### General input

In [14]:
grd_file = "Grid_001.grd"
fpv_shp_file = "fpv/fpv.shp" # it must be in the same coordinate reference system as the grd file

# meteo files parameters
first_data_value = "grid_llcorner" # see Delft3D manual; you may check the resultant files with QUICKPLOT
data_row = "grid_row" # see Delft3D manual; you may check the resultant files with QUICKPLOT
start_datetime = "2022-09-09 00:00:00 +01:00" # must include time zone

# number of decimal places (precision): e.g., 4 results in "3.1416e00" (always in scientific notation)
# since meteo files are text files, prec has a strong influence on file size and writing time
# keep it to the minimum necessary while maintaining accuracy (e.g., according to instrument resolution)
# the same prec will be used for all meteo files, so be careful!
prec = 4

fpv = True # if True, use FPV area for meteo spatialization; uniform meteo otherwise
label = "-sorriso" # additional info to append to the filename: e.g., "cloudinessLABEL.amu"

# attenuation factors: multiplied to the time series value to reduce it
meteo_att = {
    'relative_humidity': 1,
    'air_temperature': 1,
    'air_pressure': 1,
    'cloudiness': 1,
    'sw_radiation_flux': 0.27,
    'x_wind': 0.77,
    'y_wind': 0.77,
	'wind': 0.77
}

test_times = None # number of times to write for testing purposes; None for all

### Specific input

In [15]:
# Case-dependent: depends on the time series files provided by the user
tem_file = "meteo_varying_cc_complete.tem"
wind_file = "wind_dir_unit_converted.csv"

In [16]:
# read files and add proper names to columns
tem_df = pd.read_csv(
    tem_file,
    header = None,
    sep="\s+",
    names=['minutes', 'relative_humidity', 'air_temperature', 'cloudiness', 'sw_radiation_flux'],
)

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dact\AppData\Local\Temp\ipykernel_19260\2093141691.py:5: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+",


In [17]:
# read files and add proper names to columns
wind_df = pd.read_csv(
    wind_file,
    header = None,
    usecols=[0, 2, 3],
    names=['minutes', 'x_wind', 'y_wind']
)

In [18]:
# the actual input is the meteo_df dataframe
# it must have the exact column names in METEO_UNITS_EXT (order is not important):
# relative_humidity, air_temperature, cloudiness, sw_radiation_flux, x_wind, y_wind, air_pressure

meteo_df = pd.merge(tem_df, wind_df, on="minutes")

# add constant pressure
meteo_df["air_pressure"] = 101_300 # Pa

# uncomment the following to test the file writing with the first N lines
if test_times is not None:
    meteo_df = meteo_df[:test_times]

meteo_df['wind'] = np.sqrt(meteo_df.x_wind**2 + meteo_df.y_wind**2)

meteo_df

,minutes,relative_humidity,air_temperature,cloudiness,sw_radiation_flux,x_wind,y_wind,air_pressure,wind
0,0.0,91.30,15.88,57.439257,0.000,0.113313,-0.580036,101300,0.591
1,10.0,91.80,15.80,57.439257,0.000,-0.192027,0.137192,101300,0.236
2,20.0,91.80,15.92,57.439257,0.000,-0.346191,-0.841577,101300,0.910
3,30.0,92.10,15.88,57.439257,0.000,0.753065,-0.269987,101300,0.800
4,40.0,92.40,15.90,57.439257,0.000,0.747630,-0.469828,101300,0.883
...,...,...,...,...,...,...,...,...,...
109,1090.0,72.64,19.56,57.439257,4.915,-1.669754,2.260093,101300,2.810
110,1100.0,73.91,19.37,57.439257,1.125,-2.046437,1.506246,101300,2.541
111,1110.0,74.58,19.29,57.439257,0.000,0.654779,2.425162,101300,2.512
112,1120.0,75.53,19.24,57.439257,0.000,-2.085045,1.234308,101300,2.423


## Preliminaries

In [19]:
# use the same start datetime for all files
timestamp = "TIME = {{minutes}} minutes since {datetime}\n".format(datetime=start_datetime)

# use the same precision for all files
fmt = f"%.{prec}e"

In [20]:
x, y = read_grd_file(grd_file)

In [21]:
# define mask (all False if no FPV)
if fpv:
    ds = ogr.Open(fpv_shp_file)
    layer = ds.GetLayer(0)

    polys = []
    for i in range(layer.GetFeatureCount()):
        feature = layer.GetFeature(i)
        geom = feature.geometry().Clone()
        polys.append(geom)

    mask = is_fpv(x, y, polys)
else:
    mask = np.full_like(x, False, dtype=bool)

c:\Users\dact\miniforge3\envs\jupyter2024\Lib\site-packages\osgeo\ogr.py:601: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


## FLOW meteo file

In [13]:
for quantity1 in meteo_df.columns[1:]:
    print(quantity1, end="... ")
    unit1, ext = METEO_UNITS_EXT[quantity1]
    att = meteo_att[quantity1]
    att_mask = np.where(mask, att, 1)
    with open(f"{quantity1}{label}.{ext}", "w") as file:
        header = METEO_HEADER_TPL.format(
            grid_file=grd_file,
            first_data_value=first_data_value,
            data_row=data_row,
            quantity1=quantity1,
            unit1=unit1,
        )
        file.write(header)
        for minutes, quantity in zip(meteo_df.minutes, meteo_df[quantity1]):
            file.write(timestamp.format(minutes=minutes))
            mat = quantity * att_mask
            # much faster and concise than using file.write iteratively:
            np.savetxt(file, mat, fmt=fmt)
    print("(OK)")

relative_humidity... (OK)
air_temperature... (OK)
cloudiness... (OK)
sw_radiation_flux... (OK)
x_wind... (OK)
y_wind... (OK)
air_pressure... (OK)


## WAQ segment function file

In [52]:
def cell_center_mean(xx):
	"""xx is an array that represents the values at cell corners
	The function returns the mean values at cell centers by averaging the 4 corners of each cell."""
	yy = xx[:, :-1] + xx[:, 1:]
	zz = yy[:-1, :] + yy[1:, :]
	return zz / 4

In [ ]:
# read segment number array (mask for WAQ input file)
# segment number can be exported as .mat file from the .lga/.cco files with QUICKPLOT
segnum_mat = spio.loadmat("segment number.mat")
segnum = segnum_mat["data"][0, 0]["Val"]

In [55]:
for quantity1 in meteo_df[["sw_radiation_flux", "wind"]]:
    print(quantity1, end="... ")
    att = meteo_att[quantity1]
    att_mask = np.where(mask, att, 1)
    with open(f"waq-{quantity1}{label}.bin", "wb") as f:
        for minutes, quantity in zip(meteo_df.minutes, meteo_df[quantity1]):
            seconds = minutes * 60
            cell_values = cell_center_mean(quantity * att_mask)
            Ns, Ms = cell_values.shape
            # segnum is used as a mask for WAQ
            values_lin = np.zeros(int(np.nanmax(segnum)))
            for n in range(Ns):
                for m in range(Ms):
                    i = segnum[m + 1, n + 1, 0]
                    if not np.isnan(i):
                        i = int(i)
                        values_lin[i - 1] = cell_values[n, m]
            f.write(np.array(seconds, dtype=np.int32).tobytes())
            f.write(values_lin.astype(np.float32).tobytes())
    print("(OK)")

sw_radiation_flux... (OK)
wind... (OK)
